In [1]:
import psycopg2
import sys
import re
 
def main():
    #Define our connection string
    conn_string = "host='52.43.75.12' port='5432' dbname='ndexstats' user='ndexstats' password='dashboard101'"
 
    # print the connection string we will use to connect
    print "Connecting to database\n\t->%s" % (conn_string)
 
    # get a connection, if a connect cannot be made an exception will be raised here
    try:
        conn = psycopg2.connect(conn_string)
    except:
        e = sys.exc_info()[0]
        print "Error: %s" % e 
        sys.exit()

        
    # conn.cursor will return a cursor object, you can use this cursor to perform queries
    cursor = conn.cursor()
    print "Connected!\n"
    
    # input log file to be parsed and inserted into ndexstats table -- hardcoded for now
    inputfile = "ndex-2016-02-27.0.log"
    
    number_of_lines_processed = 0
    number_of_lines_total = 0
    
    with open(inputfile, "r") as f:
        for line in f:
            
            number_of_lines_total += 1
            
            split_line = re.split(r'\t+', line) 
            
            log_entry_split = [token.replace('[','').replace(']','').replace('\n','') for token in split_line]  
            timestamp, type, ip, id, account, log, api = log_entry_split
            
            # skip logs without Transaction IDs and DEBUG log entries 
            if (not id) or (type == 'DEBUG') :
                continue
                   
            if (type == 'ERROR'):
                error = True
            else:
                error = False
                
            start = log.startswith('start:')
            end = log.startswith('end:')
            
            if (checkIfRecordExistsInTable(cursor, id)):
                # record exists in table; update it
                updateRecordInTable(cursor, timestamp, type, ip, id, account, log, api, error, start, end)
            else:
                # record doesn't exist in table; insert it
                insertRecordInTable(cursor, timestamp, type, ip, id, account, log, api, error, start, end)
            
            number_of_lines_processed += 1
          
    print "\n finished inserting,  now calculate ..."

    # use transaction_start and transaction_end to calculate and fill transacation_duration_in_ms column
    conn.commit()
    calculateTransactionsDurations(cursor)
    
    print "\nnumber of lines total =", number_of_lines_total, ", processed =", number_of_lines_processed
    
    conn.commit()
    countAll(cursor)
        
    cursor.close()
    conn.close()
    
    
def countAll(cursor):
    cursor.execute("select count(*) from ndexstats ")
    number_of_records = cursor.fetchone()[0]
    
    print "number_of_records = ", number_of_records

        
def updateRecordInTable(cursor, timestamp, type, ip, id, account, log, api, error, start, end): 
    query = "update ndexstats set (transaction_end, log_entry_end, transaction_type) = "
    query += "(%s, %s, %s) where transaction_id = %s"
    data  = (timestamp, log, type, id)

    if (end or error):
        try:
            cursor.execute(query, data)
            
        except:
            e = sys.exc_info()[0]
            print "Error: %s" % e 
 
         
def insertRecordInTable(cursor, timestamp, type, ip, id, account, log_entry, api, error, start, end):
    query = "insert into ndexstats (transaction_id, transaction_start, transaction_type, client_ip, account, api, log_entry_start) "

    query += " values (%s, %s, %s, %s, %s, %s, %s);"
    data = (id, timestamp, type, ip, account, api, log_entry)
    
    if (start):
        try:
            cursor.execute(query, data)
        except:
            e = sys.exc_info()[0]
            print "Error: %s" % e 
    

def checkIfRecordExistsInTable(cursor, key):
    cursor.execute("select count(*) from ndexstats where transaction_id=%s", (key, ))
    number_of_records = cursor.fetchone()[0]

    if number_of_records == 0:
        return False
    else:
        return True
    
    
def calculateTransactionsDurations(cursor):
    query = "select transaction_id, \
               extract(epoch from to_timestamp(transaction_start,'YYYY-MM-DD HH24:MI:SS,MS'))*1000 as transaction_start, \
               extract(epoch from to_timestamp(transaction_end,'YYYY-MM-DD HH24:MI:SS,MS'))*1000 as transaction_end, \
               transaction_duration_in_ms \
            from ndexstats where transaction_duration_in_ms is null"
    
    cursor.execute(query)
    

    count = 0
    query = "update ndexstats set transaction_duration_in_ms = %s where transaction_id = %s"
    
    rows = cursor.fetchall()

    for row in rows:
      
        transaction_duration_in_ms = long(row[2]) - long(row[1])
        
        data  = (transaction_duration_in_ms, row[0])

        try:
            cursor.execute(query, data)
            
        except:
            e = sys.exc_info()[0]
            print "Error: %s" % e 
        
        count += 1

    print "updated ", count, " records"  

    
    
if __name__ == "__main__":
    main()

Connecting to database
	->host='52.43.75.12' port='5432' dbname='ndexstats' user='ndexstats' password='dashboard101'
Connected!


 finished inserting,  now calculate ...
printed  253  times

number of lines total = 692 , processed = 506
number_of_records =  253
